In [ ]:
import cv2
import numpy as np
from PIL import Image
from datasets.load import load_dataset
from pycocotools import mask as maskUtils

In [ ]:
dataset = load_dataset('coco_detection', trust_remote_code=True)

In [ ]:
def transforms(examples):
    inst_rles = examples.pop('inst.rles')
    annotation = []
    for i in inst_rles:
        inst_rles = [
            {'size': size, 'counts': counts}
            for size, counts in zip(i['size'], i['counts'])
        ]
        annotation.append(maskUtils.decode(inst_rles))
    examples['annotation'] = annotation
    return examples

def visualize(example, names, colors):
    image = np.array(example['image'])
    bboxes = np.array(example['bboxes']).round().astype(int)
    bboxes[:, 2:] += bboxes[:, :2]
    categories = example['categories']
    masks = example['annotation']
    n = len(bboxes)
    for i in range(n):
        c = categories[i]
        color, name = colors[c], names[c]
        cv2.rectangle(image, bboxes[i, :2], bboxes[i, 2:], color.tolist(), 2)
        cv2.putText(
            image, name, bboxes[i, :2], cv2.FONT_HERSHEY_SIMPLEX,
            1, color.tolist(), 2, cv2.LINE_AA, False
        )
        image[masks[..., i] == 1] = image[masks[..., i] == 1] // 2 + color // 2
    return image


dataset.set_transform(transforms)

names = dataset['train'].features['categories'].feature.names

colors = np.ones((80, 3), np.uint8) * 255
colors[:, 0] = np.linspace(0, 255, 80)
colors = cv2.cvtColor(colors[None], cv2.COLOR_HSV2RGB)[0]

example = dataset['train'][500]
Image.fromarray(visualize(example, names, colors))